In [3]:
file_location=None

if not file_location:
    file_location= '/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv'
    
primary_key = ['Process','Categorization','Word']
    
results_df = pd.read_csv(file_location)

In [4]:
results_df[['Word','Historical_Score'].

,Date,Temp_Score,Historical_Score,Word,Process,Categorization,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,2026-01-07,0,-3,SGDClassifier,ML Model,Algorithm,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
1,2026-01-07,0,-3,VotingClassifier,ML Model,Algorithm,VotingClassifier is an ensemble method that co...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Classification,Meta-Model,1.0
2,2026-01-07,0,-3,Quantile Random Forest,ML Model,Algorithm,Quantile Random Forest is an extension of rand...,NaN,NaN,NaN,NaN,NaN,Supervised,Regression,Tree,NaN
3,2026-01-07,0,-3,RandomTreesEmbedding,ML Model,Algorithm,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
4,2026-01-07,0,-3,Area Under the Curve,Model Evaluation,Feature Selection,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,0,-3,Perceptron,ML Model,Algorithm,The Perceptron is one of the earliest linear c...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
6,2026-01-08,0,-3,RidgeCV,ML Model,Algorithm,RidgeCV is a regression model that combines ri...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Regression,Linear,NaN
7,2026-01-08,0,-3,Regularization,ML Model,Definition,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,0,-3,SelectFpr,ML Model,Algorithm,SelectFpr is a feature selection method that s...,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
9,2026-01-08,0,-3,AdaBoostClassifier,ML Model,Algorithm,AdaBoostClassifier is an ensemble boosting alg...,NaN,NaN,NaN,NaN,"Small, Medium",Supervised,Classification,Linear,1.0


In [2]:
def review_test_results(file_location=None):

    '''
    Function to Facilitate a Daily Review of Historically Created Words. 
    Function has a scoring Component, a Correct Answer (Pass is worth 1 Point), A Incorrect Answer (Fail is worth -2 points), if cummulative score is not postive
    then user is expected to Answer, with expectation of making score positive, if less than 10 examples with a negative score, it randomly samples from 
    positive scores.

    
    '''

    if not file_location:
        file_location= '/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv'
    
    primary_key = ['Process','Categorization','Word']
    
    results_df = pd.read_csv(file_location)
    
    results_df["Date"] = pd.to_datetime(results_df["Date"], format='%Y-%m-%d')
    results_df["Date"] = results_df["Date"].apply(lambda x:x.date())
    
    # Generate Data Set to Test. 
    # Test Everything where Review_Score is 0.
    review_df = results_df[(results_df['Historical_Score']<0)]

    # If Less than 10 Items to review then sample historical items 
    if len(review_df)<10:
        review_df = pd.concat([
            review_df,
            results_df[(results_df['Historical_Score']>0)].sample(10-len(review_df))
        ])
        
    review_df = review_df.reset_index(drop=True)

    results_dict = {}
    for count in range(len(review_df)):
        cat,cat1,word = review_df[primary_key].iloc[count]    
        print(f"Process: {cat}\nClassification: {cat1}\nWord: {word}\n")
        print("#############################################################################################################################")
        result = input('Did you Pass or Fail?')
        results_dict[word] = [1 if result.lower() =='p' else -2][0] 
        df, nt,lk,md,ds, lt,ac = review_df.iloc[count][['Definition','Notes','Link','Markdown Equation','Dataset Size','Learning Type',"Algorithm Class"]]
        print(f"Definition: {df}\nNotes: {nt}\nLink: {lk}\nMarkdown: {md}\nDataset Size: {ds}\nLearning Type: {lt}\nAlogrithm Class: {ac}\n")

    results_df['Temp_Score'] = results_df['Word'].map(results_dict).fillna(0)
    results_df['Historical_Score'] = results_df['Historical_Score'] + results_df['Temp_Score'] 
    results_df['Temp_Score'] = 0

    results_df.to_csv(file_location,index=False)
    
    return results_df

In [33]:
import os
os.listdir('')

['.DS_Store',
 'Datasets.ipynb',
 'Pokemon',
 '.ipynb_checkpoints',
 'daily_test_results.csv']

In [34]:
df["Process_Count"] = df.groupby("Process")["Process"].transform("count")
df["CAT_Count"] = df.groupby("Categorization")["Categorization"].transform("count")
df["Word_Count"] = df.groupby("Word")["Word"].transform("count")
df['ProcessCAT_Count'] = df.groupby(['Process','Categorization'])["Word"].transform("count")

df.to_csv('/Users/derekdewald/Documents/Python/Github_Repo/Data/Streamlit_DefinitionSummary.csv',index=False)

In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [4]:
from data_d_strings import gpt_question

gpt_question(['Davies–Bouldin','Calinski–Harabasz'])

For the following words, can you please provide me for each word a definition which is 4-5 sentences highlighting the definition,
its origin, its importance and application. If the word is a ML Model, can you also please define the following in point form,
type of ML learning, type of ML problem, size of data set it works best on, and anything else which might be appropriate.
Unique Words: Davies–Bouldin, Calinski–Harabasz
    


In [ ]:

1. Correlation & Redundancy Analysis
    1. If Greater than .9 then redudanant, as a rule of thumb, newest is best, but can be based on decision
2. Variance & Stability Contribution
    1. Variance Inflation Factor
3. Clustering Sensitivity / Stability Tests
    1. Testing Results of Model
    2. Adjusted Rand Index (ARI)
    3. Silhouette/ DB Index
    4. Run Now and Overtime
4. Incremental Value via Feature Importance in Unsupervised Models
    1. Reconstruction Error in Autoencoders
    2. Mutual Information with Latent Cluster Assignment
    3. Permutation for Clustering.


Mechanism
Structure
Action and Learning.

Purpose and Contraints.
Stabiltiy and Change. 

Causation and Correlation. We're just looking at Correlation. Not Causation

Clustering utilizes mathematical precision to identify 

## Need Process for Clustering to Document decision on inclusion as it tests and score tests results somewhere central with rationale

1. Identify and validate the behavioural attributes, transactions, and usage patterns that define similarity among members
2. Understand the relative contribution and sensitivity of features in forming and separating clusters
3. Identify and define distinct, stable member archetypes based on observed behavioural patterns
4. Characterize each archetype using clear, interpretable behavioural and financial characteristics
5. Define the intended use, boundaries, and decision domains for archetype-based strategies
6. Design and apply differentiated engagement strategies aligned to each archetype’s characteristics and needs
7. Monitor archetype-level outcomes, migration, and engagement effectiveness over time

If this project succeeds, you should be able to answer:
* “Which archetype are we intentionally trying to grow?”
* “Which archetype are we comfortable shrinking?”
* “Which engagement changed member behavior within a cluster?”
* “Which engagement caused members to migrate to a better archetype?”



In [ ]:


Jan 7
Kmeans Clustering, Parameter = n_init='auto' KMeans is sensitive to initial centroid positions. A bad initialization can lead to poor clustering (high inertia). To mitigate this, KMeans runs multiple times with different random initializations and picks the best run (lowest inertia)

Interia

The Davies–Bouldin Index measures clustering quality by evaluating the average similarity between each cluster and its most similar cluster, based on the ratio of within-cluster scatter to between-cluster separation. Lower values indicate better clustering because clusters are compact and well-separated. It is useful when comparing different clustering models or parameter settings, especially for algorithms like K-Means. Expected values: closer to 0 is better, and there is no fixed upper bound—higher values suggest overlapping or poorly separated clusters.

The Calinski–Harabasz Index (Variance Ratio Criterion) assesses clustering quality by comparing the dispersion between clusters to the dispersion within clusters. Higher values indicate better-defined clusters because they are more separated and internally cohesive. It is commonly used for model selection, such as choosing the optimal number of clusters in K-Means or hierarchical clustering. Expected values: larger is better, and the score tends to increase with more distinct clusters and decrease when clusters overlap

It calculates the ratio of the sum of squared distances between cluster centers (between-group variance) to the sum of squared distances within clusters (within-group variance), scaled by the number of clusters and points.


< 1.0 → Generally considered good clustering.
1–2 → Acceptable but could be improved.
> 2 → Often suggests poor separation or overlapping clusters.


# Wall Time Versus Process Time
time.process_time()
time.perf_counter()

##########################################################################
Jan 6
- Clarify what Clustering is. 
    - Which Features Create a Stable, Interpretable and Useful Segment. 
    - Not which Features are most important.
- Pipeline - Drop Characteristics.
- Change Cluster Size.
- Articulate Meaning of Clusters and Explain Grouping. 
- How do Edge Cases Impact.
- Do I need to break up my Definitions based on Time. TO include Shorter and Longer Term.
- How can I quantify, measure and explain Clusters.

The right way to think about “sweet spot”
The “sweet spot” is not:
* maximum variance explained
* maximum number of features
* maximum complexity
It is:
* stable clusters
* interpretable differences
* consistent behavior across runs
* sensitivity that matches your use case (short-term vs long-term behavior)

Step 1 — Group features into families (critical)
Never drop features one-by-one randomly. Drop feature families together.

Ratios are:
* high signal
* high risk
They:
* amplify noise
* can dominate PCA
* are hard to interpret
Rule:
* Try models with and without ratios
* Prefer simpler models unless ratios clearly add value
Ratios must earn their place.

Balanced behavioral model
* 3M avg
* 6M growth
* engagement
* transaction mix
Long-term structure model
* 6–12M avg
* 12M growth
* low short-term noise
Short-term sensitivity model
* 1–3M dynamics
* volatility
* no long-term averages


Silhouette: Is this point closer to points in its own cluster than to points in the nearest other cluster 
Cohesion (how tight your cluster is)
Separation (how far apart clusters are)

For K Means. 
Does this variable differ meaningfully between clusters?

Fit Predict, You get one thing only:
The index of the nearest centroid for each observation
That’s it.
Important clarifications:
* It does not tell you how confident the assignment is
* It does not tell you how far a point is from its centroid
* It does not tell you how ambiguous the assignment was




In [3]:


# --- Example usage ---
# 1) Keep only numeric columns
num_cols = cc_model_df.select_dtypes(include=[np.number]).columns

# 2) (Recommended) Standardize
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(cc_model_df[num_cols]), columns=num_cols, index=cc_model_df.index)

# 3) Compute VIF
vif_table = compute_vif(X_scaled)
print(vif_table)

### Reviewing Variables for Model Inclusion

def set_manual_column_order(df,column_name,column_order):
    
    '''
    
    '''
    
    df[column_name] = pd.Categorical(df[column_name], categories=column_order, ordered=True)

import timeit
from UtilityFunctions import PauseProcess
from eda_dq import column_statistical_review

def data_preparation_checklist(df=pd.DataFrame(),word_list=[]):
    if len(df)==0:
        df = pd.read_csv(links['d_learning_notes_url'])

    final_df = pd.DataFrame()
    
    for word in word_list:
        temp_df = df[(df['Definition'].fillna("").str.contains(word,case=False))|(df['Categorization']==word)]
        final_df = pd.concat([final_df,temp_df])

    final_df = final_df.drop(['Source','Process','Categorization'],axis=1)
    display(final_df)

    return final_df

def build_member_snapshot(final_mbr_df: pd.DataFrame,
                          final_txn_df: pd.DataFrame,
                          snapshot_month=None,
                          window_3m=3,
                          window_6m=6) -> pd.DataFrame:
    m = final_mbr_df.copy()
    t = final_txn_df.copy()

    # normalize month to month-start timestamp
    m["MONTH"] = _month_floor(m["MONTH"])
    t["MONTH"] = _month_floor(t["MONTH"])

    if snapshot_month is None:
        snapshot_month = m["MONTH"].max()
    snapshot_month = pd.to_datetime(snapshot_month).to_period("M").to_timestamp()

    # restrict to history up to snapshot month
    m = m[m["MONTH"] <= snapshot_month]
    t = t[t["MONTH"] <= snapshot_month]

    # ----- Member-month features windows -----
    # Keep a lean set of signals from your schema
    base_cols = [
        "MEMBERNBR","MONTH","AGE","CITY","BRANCHNAME","PRIMARY_IS_BEEM","HOME_OWNER",
        "ANNUAL_INCOME","DEPOSIT_BALANCE","MORTGAGE_BALANCE","LIQUID_ASSETS",
        "ENGAGEMENT_SCORE","PERC_DEPOSIT","PERC_MORTGAGE",
        "POS_TXN_DEBIT","BILL_PAY_DEBIT","PAYROLL_DEPOSIT","OTHER_DEPOSIT"
    ]
    base_cols = [c for c in base_cols if c in m.columns]
    m = m[base_cols].sort_values(["MEMBERNBR","MONTH"])

    # helper: window filter
    start_3m = snapshot_month - pd.DateOffset(months=window_3m-1)
    start_6m = snapshot_month - pd.DateOffset(months=window_6m-1)

    m3 = m[m["MONTH"].between(start_3m, snapshot_month)]
    m6 = m[m["MONTH"].between(start_6m, snapshot_month)]

    # 3m aggregates (means/sums)
    agg_3m = m3.groupby("MEMBERNBR").agg(
        age=("AGE","last"),
        city=("CITY","last"),
        branch=("BRANCHNAME","last"),
        primary=("PRIMARY_IS_BEEM","last"),
        home_owner=("HOME_OWNER","last"),
        income_mean_3m=("ANNUAL_INCOME","mean"),
        deposit_mean_3m=("DEPOSIT_BALANCE","mean"),
        loan_mean_3m=("MORTGAGE_BALANCE","mean"),
        engagement_mean_3m=("ENGAGEMENT_SCORE","mean"),
        pos_sum_3m=("POS_TXN_DEBIT","sum"),
        bill_sum_3m=("BILL_PAY_DEBIT","sum"),
        payroll_sum_3m=("PAYROLL_DEPOSIT","sum"),
        otherdep_sum_3m=("OTHER_DEPOSIT","sum"),
    ).reset_index()

    # 6m dynamics: volatility + trend (slope)
    def dyn_features(df, col):
        return df.groupby("MEMBERNBR")[col].agg(
            **{f"{col.lower()}_std_6m":"std",
               f"{col.lower()}_slope_6m":_slope}
        )

    dep_dyn = m6.groupby("MEMBERNBR")["DEPOSIT_BALANCE"].agg(
        deposit_std_6m="std",
        deposit_slope_6m=_slope
    )
    loan_dyn = m6.groupby("MEMBERNBR")["MORTGAGE_BALANCE"].agg(
        loan_std_6m="std",
        loan_slope_6m=_slope
    )
    eng_dyn = m6.groupby("MEMBERNBR")["ENGAGEMENT_SCORE"].agg(
        engagement_std_6m="std",
        engagement_slope_6m=_slope
    )

    dyn = pd.concat([dep_dyn, loan_dyn, eng_dyn], axis=1).reset_index()

    # ----- Transaction mix features (from your long txn table) -----
    # Build 3m spend mix + entropy (behavioral fingerprint)
    t3 = t[t["MONTH"].between(start_3m, snapshot_month)].copy()
    # aggregate category totals per member
    cat = t3.groupby(["MEMBERNBR","variable"])["value"].sum().reset_index()
    total = cat.groupby("MEMBERNBR")["value"].sum().rename("txn_total_3m").reset_index()
    cat = cat.merge(total, on="MEMBERNBR", how="left")
    cat["share"] = np.where(cat["txn_total_3m"] > 0, cat["value"] / cat["txn_total_3m"], 0.0)

    # pivot shares wide: share_<category>
    mix = cat.pivot_table(index="MEMBERNBR", columns="variable", values="share", fill_value=0.0)
    mix.columns = [f"share_{c}" for c in mix.columns]
    mix = mix.reset_index()

    # entropy: higher = more diverse mix
    # (avoid log(0) by masking)
    share_mat = mix.drop(columns=["MEMBERNBR"]).to_numpy(dtype=float)
    with np.errstate(divide="ignore", invalid="ignore"):
        ent = -(share_mat * np.log(np.where(share_mat > 0, share_mat, 1.0))).sum(axis=1)
    mix["mix_entropy_3m"] = ent

    # ----- Combine -----
    snap = agg_3m.merge(dyn, on="MEMBERNBR", how="left").merge(mix, on="MEMBERNBR", how="left")

    # convenience derived features
    snap["net_worth_mean_3m"] = snap["deposit_mean_3m"].fillna(0) - snap["loan_mean_3m"].fillna(0)
    snap["snapshot_month"] = snapshot_month

    # fill missing mix features with 0 (e.g., no txns in window)
    share_cols = [c for c in snap.columns if c.startswith("share_")]
    snap[share_cols] = snap[share_cols].fillna(0.0)
    for c in ["deposit_std_6m","deposit_slope_6m","loan_std_6m","loan_slope_6m",
              "engagement_std_6m","engagement_slope_6m","mix_entropy_3m"]:
        if c in snap.columns:
            snap[c] = snap[c].fillna(0.0)

    return snap


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline

def make_preprocess_pipeline(feature_df: pd.DataFrame,
                             cat_cols=("city","branch"),
                             force_drop=("MEMBERNBR","snapshot_month")):
    df = feature_df.copy()

    # numeric candidates: everything number-ish except ids
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols = [c for c in num_cols if c not in force_drop]

    cat_cols = [c for c in cat_cols if c in df.columns]

    # log-transform for heavy-tailed money-ish cols
    log_cols = [c for c in num_cols if any(k in c for k in ["deposit","loan","income","pos_","bill_","payroll","otherdep","net_worth"])]
    passthrough_cols = [c for c in num_cols if c not in log_cols]

    log_pipe = Pipeline([
        ("log1p", FunctionTransformer(lambda x: np.log1p(np.clip(x, 0, None)))),
        ("scaler", RobustScaler())
    ])

    num_pipe = Pipeline([
        ("scaler", RobustScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("log_num", log_pipe, log_cols),
            ("num", num_pipe, passthrough_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ],
        remainder="drop"
    )
    return pre

# final_mbr_df, final_txn_df = generate_synthetic_dataset(...)

snap = build_member_snapshot(final_mbr_df, final_txn_df, snapshot_month=None, window_3m=3, window_6m=6)

feature_snap = snap.drop(columns=['MEMBERNBR','snapshot_month'],errors='ignore')
member_ids = snap['MEMBERNBR'].to_numpy()

pre = make_preprocess_pipeline(feature_snap)
X = pre.fit_transform(feature_snap)

print(X.shape)

# Cant run because it errors out, would be 22 Billion . Need to Understand this
#S = cosine_similarity(X)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=51,metric='cosine',algorithm='auto')

nn.fit(X)

member_index = {mid: i for i, mid in enumerate(member_ids)}

def test_knn(X,nn,k=10,sample_size=100,seed=42):
    rng = np.random.default_rng(seed)
    idxs = rng.choice(X.shape[0],size=sample_size,replace=False)

    # returns the distance and Index
    nn_dist,nn_index = nn.kneighbors(X[idxs],n_neighbors=k+1)
    nn_dist = nn_dist[:,1:].mean(axis=1)
    
    rand_idxs = rng.choice(X.shape[0],size=(sample_size,k),replace=True)
    rand_dist = np.linalg.norm(X[idxs][:,None,:]-X[rand_idxs],axis=2).mean(axis=1)
    
    return nn_dist,rand_dist,idxs


a,b,c = test_knn(X,nn)

mid = int(member_ids[0])
neighbors, sim = knn_topk(nn, member_index, member_ids, X, mid, k=10)
list(zip(neighbors.tolist(), sim.tolist()))


##########################################################################

# What does StandardScaler do. Transforms data to Mean 0 and Standard Dev = 1\

def Heatmap(df,
            correlation=True,
            column_list=[],
            title='Heat Map of Correlation',
            cmap='coolwarm',
            annotate=True,
            x_rotate=0,
            y_rotate=0,
            cbar=True,
            set_center=0,
            figsize=(10,10)):
    
    '''
    Function Which Generates a Heatmap
    
    Parameters:
        Dataframe
        column_name (list): If included, will only show certain columns on the Horizontal Axis.
    
    Returns:
        matlplot plot.
    
    '''
    
    sns.set(style='white')
    
    # View column with Abbreviated title or full. Abbreviated displays nicer.
    if correlation:
        corr = df.corr()
    else:
        corr = df.copy()
    
    if len(column_list)!=0:
        corr = corr[column_list]
    
    mask= np.zeros_like(corr,dtype=bool)
    mask[np.triu_indices_from(mask)]=True
    f,ax = plt.subplots(figsize=figsize)
    
    if len(str(set_center))!=0:
        sns.heatmap(corr,mask=mask,cmap=cmap,center=set_center,square=True,linewidths=1,annot=annotate,cbar=cbar)
    else:
        sns.heatmap(corr,mask=mask,cmap=cmap,square=True,linewidths=1,annot=annotate,cbar=cbar)
    
    
    plt.title(title)
    if y_rotate !=0:
        for tick in ax.get_yticklabels():
            tick.set_rotation(0)
            tick.set_horizontalalignment('right')
    if x_rotate !=0:
        plt.xticks(rotation=x_rotate,ha='center', va='top')

    plt.show()
